In [27]:
pip install requests beautifulsoup4


In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata

URL = "https://www.huila.gov.co/cultura-y-turismo/publicaciones/6130/patrimonio-tangible-inmueble/"

def norm_txt(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    return " ".join(s.split()).strip().lower()

resp = requests.get(URL, headers={"User-Agent": "Mozilla/5.0"})
resp.encoding = "utf-8"
soup = BeautifulSoup(resp.text, "html.parser")

tables = pd.read_html(resp.text)

registros = []
for df in tables:
    categoria_actual = None
    for r in df.itertuples(index=False):
        row_vals = [str(x) if pd.notna(x) else "" for x in r]
        row_norm = [norm_txt(v) for v in row_vals]
        row_join = " ".join(row_norm)

        # Detectar fila con categoría
        if any("categoria" in v for v in row_norm):
            for val in row_vals:
                if val.strip() != "":
                    categoria_actual = val.strip()
                    break
            continue

        # Saltar encabezados
        if ("municipio" in row_join and "denominacion" in row_join) or ("municipio" in row_join and "decreto" in row_join):
            continue

        # Filas con 3 valores válidos
        valores_utiles = [v.strip() for v in row_vals if v.strip() != ""]
        if len(valores_utiles) == 3:
            municipio, denominacion, decreto = valores_utiles
            registros.append({
                "Categoria": categoria_actual,
                "Municipio": municipio,
                "Denominacion": denominacion,
                "Decreto": decreto
            })


/tmp/ipython-input-1132986289.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


In [29]:
df_final = pd.DataFrame(registros)
df_final["Departamento"] = "Huila"
df_final.to_csv("patrimonio_huila.csv", index=False, encoding="utf-8")
print(f"✅ Archivo generado con {len(df_final)} registros")

✅ Archivo generado con 84 registros


In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata
import time

BASE_URL = "https://www.huila.gov.co"
URL = "https://www.huila.gov.co/cultura-y-turismo/publicaciones/9885/museos-departamento-del-huila/"

def norm_txt(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    return " ".join(s.split()).strip()

resp = requests.get(URL, headers={"User-Agent": "Mozilla/5.0"})
resp.encoding = "utf-8"
soup = BeautifulSoup(resp.text, "html.parser")

# --- Extraer lista de museos y sus links ---
museos = []
for a in soup.select("#infoPrincipal a[href]"):
    nombre = norm_txt(a.get_text())
    link = a["href"].strip()
    if link.startswith("http"):
        url = link
    else:
        url = BASE_URL + link
    museos.append({"Nombre": nombre, "URL": url})

# --- Para cada museo, entrar y extraer info ---
detalles = []
for m in museos:
    print(f"⏳ Procesando {m['Nombre']} -> {m['URL']}")
    info = ""
    try:
        r = requests.get(m["URL"], headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
        r.encoding = "utf-8"
        s = BeautifulSoup(r.text, "html.parser")

        # Extraer todos los <p> con texto
        parrafos = [norm_txt(p.get_text(" ", strip=True)) for p in s.select("div.modContent p") if p.get_text(strip=True)]
        info = " | ".join(parrafos)

    except Exception as e:
        print(f"⚠️ Error con {m['Nombre']}: {e}")
        # Se conserva el nombre, pero la info queda vacía

    detalles.append({
        "Nombre": m["Nombre"],
        "URL": m["URL"],
        "Info": info
    })
    time.sleep(1)  # pequeña pausa para no saturar el servidor
df_museos = pd.DataFrame(detalles)
print("------",df_museos.head())

⏳ Procesando Inicio -> https://www.huila.gov.co/cultura-y-turismo/
⏳ Procesando Compartir -> https://www.huila.gov.co#
⏳ Procesando Buscar -> https://www.huila.gov.co#
⏳ Procesando Facebook -> https://www.facebook.com/sharer/sharer.php?u=https://www.huila.gov.co/cultura-y-turismo/publicaciones/9885/museos-departamento-del-huila/
⏳ Procesando Twitter -> https://twitter.com/intent/tweet?url=https://www.huila.gov.co/cultura-y-turismo/publicaciones/9885/museos-departamento-del-huila/
⏳ Procesando Linkedin -> https://www.linkedin.com/shareArticle?mini=true&url=https://www.huila.gov.co/cultura-y-turismo/publicaciones/9885/museos-departamento-del-huila/
⏳ Procesando Whatsapp -> https://www.huila.gov.cowhatsapp://send?text=https://www.huila.gov.co/cultura-y-turismo/publicaciones/9885/museos-departamento-del-huila/
⚠️ Error con Whatsapp: HTTPSConnectionPool(host='www.huila.gov.cowhatsapp', port=443): Max retries exceeded with url: /send?text=https://www.huila.gov.co/cultura-y-turismo/publicacio

In [31]:

# --- Eliminar los primeros 16 registros ---
df_museos = df_museos.iloc[16:-2].reset_index(drop=True)

# Crear un diccionario con los municipios identificados según el nombre del museo
municipios_dict = {
    "Casa Museo Juan Friede": "San Agustín",
    "Museo Arqueologico": "Neiva",
    "Museo Arqueologico Villa de la Gaitana Timana": "Timaná",
    "Museo de Arte Contemporaneo del Huila": "Neiva",
    "Museo Geologico del Petroleo": "Neiva",
    "Casa Museo Sombrero de suaza": "Suaza",
    "Sala de Museo Jorge Villamiel Cordovez":"Neiva",
    "Museo Arte Contemporaneo del Huila":"Neiva",
    "Museo Paleontologico de Villavieja": "Villavieja",
    "Casa Museo Artesanal Casa del Totumo": "Villavieja",
    "Museo Parque Arqueologico Obando": "San Agustín",
    "Casa Museo Luis Duque Gomez": "Isnos",
    "Museo Prehistorico Una Mirada al Pasado": "Neiva",
    "Sala Museo Jorge Villamil Cordovez": "Neiva",
    "Museo Historia Natural de la Tatacoa": "Villavieja",
    "Museo Comunitario La Tormenta": "Villavieja"
}

# Normalizar nombres para hacer la asignación más robusta
def asignar_municipio(nombre):
    for key in municipios_dict:
        if key.lower() in nombre.lower():
            return municipios_dict[key]
    return "Desconocido"

# Agregar columnas
df_museos["Departamento"] = "Huila"
df_museos["Categoria"] = "Categoria Museos"
df_museos["Municipio"] = df_museos["Nombre"].apply(asignar_municipio)

In [32]:
df_museos.to_csv("museos_huila.csv", index=False, encoding="utf-8")
print(f"✅ Archivo generado con {len(df_museos)} museos")

✅ Archivo generado con 9 museos


In [40]:
df2_renamed = df_final.rename(columns={"Denominacion": "Nombre"})

# Realizar un merge externo (outer) para no perder información de ninguno de los dos
df_merged = pd.merge(df_museos, df2_renamed, on=["Nombre", "Municipio", "Categoria", "Departamento"], how="outer")

In [41]:
df_merged.to_csv("Lugares_Huila.csv", index=False, encoding="utf-8")
print(f"✅ Archivo generado con {len(df_merged)} lugares")

✅ Archivo generado con 93 lugares
